# Comparação de Gradient Boosting com e sem features quânticas

Este notebook carrega os *folds* disponíveis na pasta `features/`, treina modelos de Gradient Boosting usando apenas as features clássicas (`class_0` a `class_12`) e o conjunto combinado de features clássicas + quânticas (`qf_0` a `qf_12`). Em seguida, comparamos o desempenho entre os dois cenários e geramos as predições solicitadas.


## Dependências

O notebook utiliza `pandas`, `numpy`, `scikit-learn`, `matplotlib` e `seaborn`. Caso ainda não as tenha instalado no seu ambiente, execute o comando abaixo em uma célula separada ou diretamente no terminal:

```bash
pip install pandas numpy scikit-learn matplotlib seaborn
```


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    balanced_accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)

# Configuração estética padrão para os gráficos
sns.set_style("whitegrid")
plt.rcParams.update({"figure.figsize": (10, 5), "axes.titlesize": 14, "axes.labelsize": 12})


## Carregamento dos dados

Cada CSV em `features/` corresponde a um *fold*. O conjunto possui uma coluna `set` indicando se a amostra pertence ao treino ou ao teste.


In [ ]:
data_dir = Path("features")
fold_paths = sorted(data_dir.glob("features_y_fold*.csv"))
if not fold_paths:
    raise FileNotFoundError("Nenhum arquivo `features_y_fold*.csv` foi encontrado na pasta `features/`.")

fold_frames = []
for path in fold_paths:
    frame = pd.read_csv(path)
    frame["fold_name"] = path.stem
    fold_frames.append(frame)

fold_summary = (
    pd.concat(
        [df.assign(set=df["set"].str.lower())[["fold", "fold_name", "set"]] for df in fold_frames],
        ignore_index=True,
    )
    .value_counts()
    .unstack(fill_value=0)
)
fold_summary


## Definição dos grupos de features e função de avaliação

A função abaixo treina um `GradientBoostingClassifier` para cada conjunto de features em todos os *folds*, calcula as métricas desejadas no conjunto de teste e armazena as predições geradas.


In [ ]:
classical_features = [col for col in fold_frames[0].columns if col.startswith("class_")]
quantum_features = [col for col in fold_frames[0].columns if col.startswith("qf_")]

feature_sets = {
    "Benchmark": classical_features,
    "Quantum": classical_features + quantum_features,
}

metric_functions = {
    "AUC": lambda y_true, y_score, y_pred: roc_auc_score(y_true, y_score) if len(np.unique(y_true)) > 1 else np.nan,
    "F1 Score Overall": lambda y_true, y_score, y_pred: f1_score(y_true, y_pred),
    "Balanced Accuracy": lambda y_true, y_score, y_pred: balanced_accuracy_score(y_true, y_pred),
    "Precision Class 0": lambda y_true, y_score, y_pred: precision_score(y_true, y_pred, pos_label=0),
    "Precision Class 1": lambda y_true, y_score, y_pred: precision_score(y_true, y_pred, pos_label=1),
    "Recall Class 0": lambda y_true, y_score, y_pred: recall_score(y_true, y_pred, pos_label=0),
    "Recall Class 1": lambda y_true, y_score, y_pred: recall_score(y_true, y_pred, pos_label=1),
}

results = []
prediction_frames = []

for fold_idx, fold_df in enumerate(fold_frames):
    train_df = fold_df[fold_df["set"] == "train"]
    test_df = fold_df[fold_df["set"] == "test"]

    y_train = train_df["y"]
    y_test = test_df["y"]

    for label, columns in feature_sets.items():
        X_train = train_df[columns]
        X_test = test_df[columns]

        model = GradientBoostingClassifier(random_state=42)
        model.fit(X_train, y_train)

        y_proba = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)

        for metric_name, metric_fn in metric_functions.items():
            value = metric_fn(y_test, y_proba, y_pred)
            results.append(
                {
                    "fold": fold_idx,
                    "fold_name": fold_df["fold_name"].iat[0],
                    "model": label,
                    "metric": metric_name,
                    "value": value,
                }
            )

        prediction_frames.append(
            pd.DataFrame(
                {
                    "fold": fold_idx,
                    "fold_name": fold_df["fold_name"].iat[0],
                    "row_id": test_df["row_id"].values,
                    "y_true": y_test.values,
                    "model": label,
                    "y_pred": y_pred,
                    "y_proba": y_proba,
                }
            )
        )

results_df = pd.DataFrame(results)
predictions_df = pd.concat(prediction_frames, ignore_index=True)

results_df.head()


## Resumo das métricas por modelo

A tabela a seguir mostra a mediana e o intervalo interquartil (25%-75%) das métricas em todos os *folds*.


In [ ]:
metrics_summary = (
    results_df
    .groupby(['model', 'metric'])['value']
    .agg(
        median=lambda s: np.nanmedian(s),
        q1=lambda s: np.nanquantile(s, 0.25),
        q3=lambda s: np.nanquantile(s, 0.75),
    )
    .reset_index()
)
metrics_summary['iqr'] = metrics_summary['q3'] - metrics_summary['q1']
metrics_summary['median_iqr'] = metrics_summary.apply(
    lambda row: f"{row['median']:.4f} [{row['q1']:.4f}, {row['q3']:.4f}]",
    axis=1,
)
metrics_summary


## Gráfico comparativo

O gráfico reproduz o exemplo solicitado, com barras pretas representando o benchmark (apenas features clássicas) e barras amarelas representando o modelo com features clássicas + quânticas. Os valores exibidos nas barras correspondem às medianas por *fold*.


In [ ]:
metric_order = [
    'AUC',
    'F1 Score Overall',
    'Balanced Accuracy',
    'Precision Class 0',
    'Precision Class 1',
    'Recall Class 0',
    'Recall Class 1',
]

plot_ready = (
    metrics_summary
    .pivot(index='metric', columns='model', values='median')
    .reindex(metric_order)
)

ax = plot_ready.plot(
    kind='bar',
    color={'Benchmark': '#2e2e2e', 'Quantum': '#f1b82d'},
    edgecolor='black'
)

ax.set_ylim(0, 1.05)
ax.set_ylabel('Score')
ax.set_xlabel('')
ax.set_title('Toxicity classification with Gradient Boosting')
ax.legend(title='Modelo')

for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3)

plt.tight_layout()
plt.show()


## Predições

As tabelas abaixo exibem, respectivamente, as primeiras linhas das predições de teste para o modelo benchmark e para o modelo com features quânticas.


In [ ]:
benchmark_predictions = predictions_df[predictions_df['model'] == 'Benchmark']
quantum_predictions = predictions_df[predictions_df['model'] == 'Quantum']

benchmark_predictions.head()


In [ ]:
quantum_predictions.head()
